In [4]:
import requests
import pandas as pd
from datetime import date


BASE_URL = "https://statsapi.web.nhl.com"

#more API info: https://gitlab.com/dword4/nhlapi/-/blob/master/stats-api.md#people

pd.set_option("display.max_rows", None)

response = requests.get("https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster")

df = pd.json_normalize(response.json())


response.json()

c = pd.json_normalize(response.json(), record_path=["teams"])[["roster.roster"]].apply(pd.Series)

playerdf = pd.json_normalize(c.loc[25][0]).set_index("person.fullName")

personidseries = playerdf["person.id"]

player_id_df = pd.DataFrame()
df = c.reset_index()  # make sure indexes pair with number of rows

for index, row in df.iterrows():
    player_id_df = pd.concat([player_id_df, pd.json_normalize(c.loc[index][0]).set_index("person.fullName")])
#player_id_df is a df with index person.fullName 

player_id_df
# list(player_id_df.index)

# ["hello"].to_json()

,jerseyNumber,person.id,person.link,position.code,position.name,position.type,position.abbreviation
person.fullName,,,,,,,
Jonathan Bernier,45,8473541,/api/v1/people/8473541,G,Goalie,Goalie,G
Brendan Smith,2,8474090,/api/v1/people/8474090,D,Defenseman,Defenseman,D
Tomas Tatar,90,8475193,/api/v1/people/8475193,L,Left Wing,Forward,LW
Erik Haula,56,8475287,/api/v1/people/8475287,L,Left Wing,Forward,LW
Ondrej Palat,18,8476292,/api/v1/people/8476292,L,Left Wing,Forward,LW
Robbie Russo,39,8476418,/api/v1/people/8476418,D,Defenseman,Defenseman,D
Tyler Wotherspoon,26,8476452,/api/v1/people/8476452,D,Defenseman,Defenseman,D
Dougie Hamilton,7,8476462,/api/v1/people/8476462,D,Defenseman,Defenseman,D
Damon Severson,28,8476923,/api/v1/people/8476923,D,Defenseman,Defenseman,D


In [6]:
#list of player names
player_names_list = list(player_id_df.index)

In [8]:
#Player must be in the df
def getPlayerId(name):
    player_id = player_id_df.loc[name]["person.id"]
    return player_id

print(getPlayerId("Logan Couture"))

8474053


In [9]:
def get_general_stats(player_name):
    player_id = getPlayerId(player_name)
    '''
    returns df of previous stats, year to year
    and list of teams played for
    '''
    response = requests.get(BASE_URL + f"/api/v1/people/{player_id}/stats?stats=yearByYear")
    response_json = response.json()["stats"][0]["splits"]
    df = pd.json_normalize(response_json)
    teams_list = [f"{df.iloc[i]['team.name']} ({df.iloc[i]['league.name']})" for i in range(len(list(df['team.name'])) - 1, -1, -1)]
#     teams_played_for = set(df["team.name"])
    seen = set()
    teams_played_for = [x for x in teams_list if not (x in seen or seen.add(x))]
#     teams_played_for = set(teams_list)
    return df, teams_played_for

def get_stats_for_team(player_name, team, stats_df):
    df = stats_df.set_index("team.name") #where stats_df is entire player season(s) history stats
    df_team_specific = df.loc[[team]].set_index("season")
    season_list = list(df_team_specific.index)
    goal_list = list(df_team_specific["stat.goals"])
    assist_list = list(df_team_specific["stat.assists"])
    points_list = list(df_team_specific["stat.points"])
    return season_list, goal_list, assist_list, points_list, df_team_specific


player_name = "Martin Jones"

general_stats_df, teams_played_for = get_general_stats(player_name)

print(teams_played_for)
#use "teams_played_for" for drop down select
# desired_team = "Seattle Kraken"

# year_list, goal_list, assist_list, points_list, df_team_specific = get_stats_for_team(player_name, desired_team, general_stats_df)

# print(f"Year List", year_list)
# print(f"Goal List", goal_list)
# print(f"Assist List", assist_list)
# print(f"Points List", points_list)
general_stats_df

['Philadelphia Flyers (National Hockey League)', 'San Jose Sharks (National Hockey League)', 'Canada (WC)', 'Canada (WC-A)', 'Los Angeles Kings (National Hockey League)', 'Manchester (AHL)', 'Ontario (ECHL)', 'Canada (WJC-A)', 'Calgary (WHL)']


,season,sequenceNumber,stat.timeOnIce,stat.shutouts,stat.ties,stat.wins,stat.losses,stat.goalAgainstAverage,stat.games,stat.goalsAgainst,...,stat.powerPlaySaves,stat.shortHandedSaves,stat.evenSaves,stat.shortHandedShots,stat.evenShots,stat.powerPlayShots,stat.gamesStarted,stat.powerPlaySavePercentage,stat.shortHandedSavePercentage,stat.evenStrengthSavePercentage
0,20062007,1,1029:00,0.0,3.0,9.0,4.0,3.030000,18,52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20072008,1,1529:00,1.0,1.0,18.0,8.0,2.120000,27,54.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20082009,1,3295:00,7.0,4.0,45.0,5.0,2.076000,55,114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20092010,1,2851:00,8.0,1.0,36.0,11.0,2.210000,48,105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20092010,3,78:00,0.0,0.0,1.0,1.0,2.300000,2,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20102011,1,2187:00,4.0,1.0,23.0,12.0,2.250000,39,82.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,20102011,2,64:00,0.0,0.0,1.0,0.0,3.760000,1,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,20112012,1,2166:00,1.0,2.0,18.0,17.0,2.600000,41,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,20122013,1,3347:00,5.0,4.0,27.0,25.0,2.530000,56,141.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,20132014,1,1094:40,4.0,NaN,12.0,6.0,1.808770,19,33.0,...,88.0,10.0,369.0,10.0,389.0,101.0,18.0,87.128713,100.000000,94.858612


In [36]:
def getStandingsByYear(year=f"{date.today().year - 1}{date.today().year}"):
    '''
    returns standings based on year
    default is the current year
    '''
    print(f"{date.today().year - 1}{date.today().year}")
    
    
    response = requests.get(BASE_URL + f"/api/v1/standings?season={year}")
    return response.json()

getStandingsByYear("20062007")

20212022


{'copyright': 'NHL and the NHL Shield are registered trademarks of the National Hockey League. NHL and NHL team marks are the property of the NHL and its teams. © NHL 2022. All Rights Reserved.',
 'records': [{'standingsType': 'regularSeason',
   'league': {'id': 133,
    'name': 'National Hockey League',
    'link': '/api/v1/league/133'},
   'division': {'id': 1, 'name': 'Atlantic', 'link': '/api/v1/divisions/1'},
   'conference': {'id': 1, 'name': 'Eastern', 'link': '/api/v1/conferences/1'},
   'season': '20062007',
   'teamRecords': [{'team': {'id': 1,
      'name': 'New Jersey Devils',
      'link': '/api/v1/teams/1'},
     'leagueRecord': {'wins': 49, 'losses': 24, 'ot': 9, 'type': 'league'},
     'goalsAgainst': 201,
     'goalsScored': 216,
     'points': 107,
     'divisionRank': '1',
     'divisionL10Rank': '3',
     'divisionRoadRank': '1',
     'divisionHomeRank': '2',
     'conferenceRank': '2',
     'conferenceL10Rank': '7',
     'conferenceRoadRank': '2',
     'conference